# Simulation Credit Example

**[Optional for Google Colab] Installing PiML**

1. Run `!pip install piml` to install the latest version of PiML
2. In Colab, you'll need restart the runtime in order to use newly installed PiML version.

In [ ]:
! pip install piml

## Data Description

This example demonstrates the use of PiML for fairness testing. We first simulate a credit decisioning data with hypothesized features `Mortgage`, `Balance`, `Amount Past Due`, `Delinquency status`, `Credit Inquiry`, `Open Trade`, `Utilization`, as well as demographic features `Gender` and `Race`. The response `Status` is a binary indicator, and this is a classification problem. 

**Sample Size**: 60526, **Columns Num**: 10. 

**Features:**

`Mortgage` *(Numerical)*: Applicant’s mortgage size.

`Balance` *(Numerical)*: Average last 12 months credit card balance.

`Amount Past Due` *(Numerical)*: The minimum required payment that was not applied to the account as of the last payment due date.

`Delinquency Status` *(Ordinal)*: 0: current, 1: < 30 day delinquent, 2: 30-60 day delinquent, 3: 60-90 day and so on.

`Credit Inquiry` *(Ordinal)*: Number of credit inquiries in the last 12 months.

`Open Trade` *(Ordinal)*: Number of open credit accounts. 

`Utilization` *(Numerical)*: % credit utilization, the sum of all your balances, divided by the sum of your cards' credit limits.

**Demographic Features:**

Demographic features in Credit data cannot be used for modeling.

`Gender` *(Categorical)*: Two kinds of gender.

`Race` *(Categorical)*: Two kinds of race. 

**Target Response:**

`Status` *(Categorical)*: 0: default (should not be approved) and 1: non-default (should be approved). The 0/1 ratio is nearly 1:5. 

## Load and Prepare data

In [2]:
from piml import Experiment
exp = Experiment()

# Manually load data to piml
import pandas as pd
data = pd.read_csv('https://github.com/SelfExplainML/PiML-Toolbox/blob/main/datasets/CreditSimuBalanced.csv?raw=true')
exp.data_loader(data=data)

,Mortgage,Balance,Amount Past Due,Delinquency Status,Credit Inquiry,Open Trade,Utilization,Gender,Race,Status
0,135775.74,646.11,48.94,5.0,2.0,1.0,0.239912,1.0,0.0,1.0
1,168324.14,1968.77,583.77,2.0,1.0,1.0,0.204097,1.0,0.0,1.0
2,346862.01,744.26,0.00,0.0,0.0,0.0,0.483670,1.0,1.0,1.0
3,144067.40,2235.02,0.00,0.0,0.0,0.0,0.948067,0.0,0.0,0.0
4,183339.91,614.66,0.00,0.0,0.0,0.0,0.253230,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
99995,232793.63,1766.13,0.00,0.0,0.0,0.0,0.350013,1.0,1.0,1.0
99996,277526.51,1418.25,0.00,0.0,1.0,1.0,0.849044,0.0,1.0,0.0
99997,229287.59,5349.40,0.00,0.0,0.0,0.0,0.998986,1.0,1.0,1.0
99998,139738.56,697.78,0.00,0.0,0.0,0.0,0.292058,1.0,0.0,0.0


In [3]:
# Exclude features one-by-one: "Gender", "Race" (demographic variables); 
exp.data_summary()

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

HTML(value='<link rel="stylesheet" href="//stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.…

In [23]:
# Prepare dataset with Test Ratio = 0.2
exp.data_prepare()

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

In [5]:
#Feature selection with RCIT method
exp.feature_select()

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

## Model Train

Fit XGB2 and XGB2 with monotonicity.  
Monotonic increasing list: ['Mortgage', 'Balance', 'Amount Past Due'] (Feature Index:[0,1,2])   
Monotonic decreasing list: ['Utilization', 'Delinquency Status', 'Credit Inquiry', 'Open Trade']. (Feature Index:[3,4,5,6])

In [24]:
exp.model_train()

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

<IPython.core.display.Javascript object>

## Model Explainablity

1. Select the registered model "XGB2" and "XGB2_monotonic" respectively.


2. Check the partial dependence plot of `Utilization`.   

For this case, utilization should be monotonic decreasing. Howerver, without monotonicity constraint, the model explanation of XGB2 is not very smooth.

In [17]:
exp.model_explain()

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

<IPython.core.display.Javascript object>

## Fairness Testing

**Guide Step:**  

1. First select a registered model (in this case, XGB2_monotonic)  


2. Group Setting  
    - Set Add Category = "Gender", select "1.0" as reference, select "0.0" as protected, then click "Add".  
    - Set Add Category = "Race",  select "1.0" as reference, select "0.0" as protected, then click "Add".   
    
    
3. Debiasing/unfairness mitigation by Feature Binning
   - 3.1 Select AIR as fairness metric and F1 as performance metric.  
   - 3.2 Select 'Balance' as binning attribute, 'Customize' as binning method and binning range is 0-0.25.  
   - 3.3 Click the button "APPLY" to apply the binning setting to the data.   
   - 3.4 Click 'Evaluate' to compare the fairness and performance metrics after binning.  
   
   - If you want to add other binning configs to compare.
       - Click button "NEW CONFIG" to generate a new binning settting, after that repeat the 3.2-3.4 again to compare results between diffrent binning settings.
       - Click button "CLEAR ALL" could remove all the record.
   
   For this model, after binning the AIR of Gender is improved to near 1 and the performance also get tiny improved.  
   
   
4. Debiasing/unfairness mitigation by Feature Removal
   - Select AIR as fairness metric and F1 as performance metric.  
   - Check the fairness and performance metrics upon removal of one feature at a time.  

    After removing Mortgage/Balance, the AIR of Race/Gender is increased near to 1.

In [27]:
exp.model_fairness()

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

<IPython.core.display.Javascript object>

## Diagnose Testing  

**Guide Step:**


1. First select a registered model (in this case, XGB2_monotonic)  


2. Weakspot
    - Select `Mortgage` as 'Feature 1'. Choose 'ACC' as metric and set threshold to 1.0.  
    - Check the weak region plot.   
    
    For this model, the low mortgage area is weak.
    
    
3. Reliability
   - Select `Mortgage` as 'Feature'.  
   - Check the uncertain area for each config.  
   
   For this model, the low mortgage area is with high uncertainty.
   
   
4. Resilience
   - Select `Mortgage` as 'Plot Feature'. Choose 'ACC' as metric.
   - Check the distribution shift of the 20% worst samples. 

    For this model, the low mortgage area has more worst samples than the rest.

In [26]:
exp.model_diagnose()

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

<IPython.core.display.Javascript object>

## Model Comparing

**Guide Step:**

1. First select registered models (in this case, XGB2 and XGB2_monotonic)   

2. Robustness  
    - Choose 'AUC' as metric.
    - Check the model performance with perturbation on all features.   
    
    For these two models, the Monotonic XGB2 is more robust than XGB2.

In [25]:
exp.model_compare()

HTML(value='\n        <style>\n\n        .left-label {\n            width: 30%;\n        }\n\n        .card-pa…

<IPython.core.display.Javascript object>